In [1]:
import pandas as pd
from scipy import optimize
import math as math
import numpy as np
import matplotlib as mpl
from gurobipy import *
from scipy.optimize import minimize
import random as random

In [15]:

def mod(c,T):
    theta=[0.1,0.1,0.1,0.2,0.15,0.05,0.3]
    prc=[12,15,30,5,25,20,17]
    beta=[[1,0,0,0],[1,1,0,0],[1,0,1,0],[0,1,0,0],[0,0,1,0],[0,1,0,1],[0,0,0,1]]
    model=Model("airline")
    x = [0 for row in range(7)]

    for j in range(7):
        x[j] = model.addVar(lb=0, name="x[%s]" % (j))
    model.update()

    model.setObjective(quicksum(T*theta[i]*prc[i]*x[i] for i in range(7)), GRB.MAXIMIZE) #Objective to maximize revenue
    model.update()

    pi=[0 for row in range(4)]
    for j in range(4):  
        #pi=0
        for i in range(7):
            pi[j]+=(T*theta[i]*beta[i][j]*x[i]) #Capacity constraint
        model.addConstr(pi[j]-c[j],"<=",rhs=0)

    for i in range(7):
        model.addConstr(x[i],">=",rhs=0) #Limits of x
        model.addConstr(x[i],"<=", rhs=1)
    model.update()
    model.optimize()

    l=[]
    for x in model.getVars():
        l.append(x.x)
    print(l)
    return(model)

    #print(pi)

In [16]:
#T=2000
#c=[360,360,360,360]
theta=[0.1,0.1,0.1,0.2,0.15,0.05,0.3]
prc=[12,15,30,5,25,20,17]
T=2000
c=[360,360,360,360]
model=mod(c,T)


Optimize a model with 18 rows, 7 columns and 24 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 6e+02]
  Objective range [2e+03, 1e+04]
  Bounds range    [0e+00, 0e+00]
  RHS range       [1e+00, 4e+02]
Presolve removed 16 rows and 3 columns
Presolve time: 0.07s
Presolved: 2 rows, 4 columns, 5 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5820000e+04   1.210938e+00   0.000000e+00      0s
       3    2.0620000e+04   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.07 seconds
Optimal objective  2.062000000e+04
[1.0, 0.5, 0.3, 0.65, 1.0, 0.0, 0.6]


In [17]:
def dualpi(cap,T):
    theta=[0.1,0.1,0.1,0.2,0.15,0.05,0.3]
    prc=[12,15,30,5,25,20,17]
    beta=[[1,0,0,0],[1,1,0,0],[1,0,1,0],[0,1,0,0],[0,0,1,0],[0,1,0,1],[0,0,0,1]]
    decision=[]
    model=mod(cap,T) #Optimizing model
    variables=model.getConstrs() 
    pi = []
    for i in range(4):
        pi.append(variables[i].getAttr("pi"))
    # beta=[[1,1,1,0,0,0,0],[0,1,0,1,0,1,0],[0,0,1,0,1,0,0],[0,0,0,0,0,1,1]]
    alist=[]
    for i in range(7):
        temp = 0
        for j in range(4):
            temp = temp + (beta[i][j] * pi[j])
        if((temp) <= prc[i]): #Making accept reject decisions
            decision.append("Accept")
        else:
            decision.append("Reject")
    print(pi)
    return(decision)
dualpi(c,T)

Optimize a model with 18 rows, 7 columns and 24 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 6e+02]
  Objective range [2e+03, 1e+04]
  Bounds range    [0e+00, 0e+00]
  RHS range       [1e+00, 4e+02]
Presolve removed 16 rows and 3 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 5 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5820000e+04   1.210938e+00   0.000000e+00      0s
       3    2.0620000e+04   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds
Optimal objective  2.062000000e+04
[1.0, 0.5, 0.3, 0.65, 1.0, 0.0, 0.6]
[10.0, 5.0, 20.0, 17.0]


['Accept', 'Accept', 'Accept', 'Accept', 'Accept', 'Reject', 'Accept']

In [5]:
#2b
cap=[360,360,360,360]
prc=[12,15,30,5,25,20,17]
beta=[[1,0,0,0],[1,1,0,0],[1,0,1,0],[0,1,0,0],[0,0,1,0],[0,1,0,1],[0,0,0,1]]
r=[]
decision=['Accept', 'Accept', 'Accept', 'Accept', 'Accept', 'Reject', 'Accept'] #Decisions

In [8]:
def custype():
    cus_type=6
    while(cus_type==6): #Checking for demand, if type 6 is found, it is rejected
        rand=random.random()
        if rand<=theta[0]:
            cus_type=1
        elif rand<=(theta[0]+theta[1]):
            cus_type=2
        elif rand<=(theta[0]+theta[1]+theta[2]):
            cus_type=3
        elif rand<=(theta[0]+theta[1]+theta[2]+theta[3]):
            cus_type=4
        elif rand<=(theta[0]+theta[1]+theta[2]+theta[3]+theta[4]):
            cus_type=5
        elif rand<=(theta[0]+theta[1]+theta[2]+theta[3]+theta[4]+theta[5]):
            cus_type=6
        elif rand<=(theta[0]+theta[1]+theta[2]+theta[3]+theta[4]+theta[5]+theta[6]):
            cus_type=7
    #print(cus_type)
    return(cus_type)

r=[]
li=0
for a in range(100): #Calculating revenue
    cap=[360,360,360,360]
    customer=[1,2,3,4,5,7]
    prc=[12,15,30,5,25,20,17]
    li=0
    for k in range(2000):
        cus_type=custype()
        if(cus_type!=6):
            j=cus_type-1   
            if(any(capacity<=0 for capacity in cap)):#Checking if capacity is 0
                break
            else:
                #for i in (beta[j]):
                 #   if(beta[j][i]>0):
                  #      temp=i
                   #     cap[i]=cap[i]-beta[j][i]
                cap = list(map(operator.sub,cap,beta[j]))
                li+=prc[j]
        else:
            li+=0
    r.append(li)  

print(np.mean(r))
print(cus_type)
print(r)

17774.25
7
[17879, 17668, 16829, 18076, 17313, 17987, 17637, 18461, 17606, 17930, 17279, 17930, 18107, 18474, 16598, 18303, 15949, 18205, 17486, 17751, 18182, 18162, 17809, 17926, 17308, 18649, 16788, 17996, 18171, 17171, 18816, 18336, 18834, 18490, 17803, 17751, 18150, 17613, 17931, 17195, 18268, 18707, 18930, 18297, 18696, 17892, 18000, 17383, 17086, 18081, 17397, 17431, 18848, 17432, 17857, 17731, 17765, 17021, 17602, 16439, 17764, 17583, 18105, 19113, 16504, 18206, 17506, 17332, 18169, 18390, 15774, 17647, 17273, 17822, 17051, 17541, 17340, 17867, 17653, 17491, 17940, 17209, 17713, 17573, 17626, 18434, 16554, 17924, 18369, 18072, 17536, 17613, 18513, 17711, 17839, 19138, 18211, 17841, 17113, 16953]


In [ ]:
#2b, for 20 periods, similar to 2a, but accept-reject is calculated every time
def custype1(d_list):
    #cus_type=6
    while(True): #Checking for demand, if type 6 is found, it is rejected
        rand=random.random()
        if rand<=theta[0]:
            cus_type=1
        elif rand<=(theta[0]+theta[1]):
            cus_type=2
        elif rand<=(theta[0]+theta[1]+theta[2]):
            cus_type=3
        elif rand<=(theta[0]+theta[1]+theta[2]+theta[3]):
            cus_type=4
        elif rand<=(theta[0]+theta[1]+theta[2]+theta[3]+theta[4]):
            cus_type=5
        elif rand<=(theta[0]+theta[1]+theta[2]+theta[3]+theta[4]+theta[5]):
            cus_type=6
        elif rand<=(theta[0]+theta[1]+theta[2]+theta[3]+theta[4]+theta[5]+theta[6]):
            cus_type=7
        if(d_list[cus_type-1]=="Reject"):
            continue
        else:
            break
    #print(cus_type)
    return(cus_type)
rev=[]
for m in range(100):
    cap=[360,360,360,360]
    prc=[12,15,30,5,25,20,17]
    li=0
    T=2000
    decision_1=[['Accept', 'Accept', 'Accept', 'Accept', 'Accept', 'Reject', 'Accept']]
    for m in range(100):
        if(T>0):
            for i in range(20):          
                cus_type=custype1(decision_1[m])
                j=cus_type-1        
                if(any(capacity<=0 for capacity in cap)):
                    break
                else:
                    cap = list(map(operator.sub,cap,beta[j]))
                li+=prc[j]
                T-=1
            ret=mod(cap,T)
            ar=dualpi(cap,T)
            decision_1.append(ar)
        else:
            break
    rev.append(li)
    

Optimize a model with 18 rows, 7 columns and 24 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 6e+02]
  Objective range [2e+03, 1e+04]
  Bounds range    [0e+00, 0e+00]
  RHS range       [1e+00, 4e+02]
Presolve removed 16 rows and 3 columns
Presolve time: 0.08s
Presolved: 2 rows, 4 columns, 5 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5519000e+04   1.224609e+00   0.000000e+00      0s
       3    2.0239000e+04   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.09 seconds
Optimal objective  2.023900000e+04
[1.0, 0.5, 0.2777777777777778, 0.648989898989899, 1.0, 0.0, 0.5959595959595959]
Optimize a model with 18 rows, 7 columns and 24 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 6e+02]
  Objective range [2e+03, 1e+04]
  Bounds range    [0e+00, 0e+00]
  RHS range       [1e+00, 4e+02]
Presolve removed 16 rows and 3 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 5 nonzeros

Iteration    Objective

In [20]:
np.mean(rev)

17923.400000000001

In [26]:
#2c, the total period is 1500, 75*20, and the rest 500 is done without making decisions
revenue=[]
for m in range(100):
    cap=[360,360,360,360]
    prc=[12,15,30,5,25,20,17]
    li=0
    T=2000
    decision_2=[['Accept', 'Accept', 'Accept', 'Accept', 'Accept', 'Reject', 'Accept']]
    for m in range(75):
        if(T>0):
            for i in range(20):          
                cus_type=custype1(decision_2[m])
                j=cus_type-1        
                if(any(capacity<=0 for capacity in cap)):
                    break
                else:
                    cap = list(map(operator.sub,cap,beta[j]))
                li+=prc[j]
                T-=1
            returned=mod(cap,T)
            ar=dualpi(cap,T)
            decision_2.append(ar)
        else:
            break
    revenue.append(li)
    
    for m in range(500):
        cus_type=custype(decision_2[m])
        j=cus_type-1 
        if(any(capacity<=0 for capacity in cap)):
             break
        else:
            cap = list(map(operator.sub,cap,beta[j]))
        li+=prc[j]
        revenue.append(li)
             
        
    

Optimize a model with 18 rows, 7 columns and 24 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 6e+02]
  Objective range [2e+03, 1e+04]
  Bounds range    [0e+00, 0e+00]
  RHS range       [1e+00, 4e+02]
Presolve removed 16 rows and 3 columns
Presolve time: 0.02s
Presolved: 2 rows, 4 columns, 5 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5514000e+04   1.205078e+00   0.000000e+00      0s
       3    2.0284000e+04   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.03 seconds
Optimal objective  2.028400000e+04
[1.0, 0.5252525252525253, 0.2777777777777778, 0.6338383838383839, 1.0, 0.0, 0.5959595959595959]
Optimize a model with 18 rows, 7 columns and 24 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 6e+02]
  Objective range [2e+03, 1e+04]
  Bounds range    [0e+00, 0e+00]
  RHS range       [1e+00, 4e+02]
Presolve removed 16 rows and 3 columns
Presolve time: 0.01s
Presolved: 2 rows, 4 columns, 5 nonzeros

Iterat

TypeError: custype() takes no arguments (1 given)

In [24]:
np.mean(revenue)

17865.009999999998